# [MVTorch](https://github.com/ajhamdi/mvtorch) 3D Calssification Example

## Setup

- download common 3D datasets ([ModelNet40](https://mega.nz/file/mm5FhJ7I#jGECWn-QSCLH9LLoxhZzSWnf9LCtCavV12toj9SJKPM), [ScanObjectNN](https://mega.nz/file/ampg2QyT#Exo22r-8jzgCa2MOqoqipd39HVqYKG5iykJ5bovjsuI)) and unzip inside `data` directory.

- Conda env : `mvtorchenv1`

In [1]:
!cd .. && cd .. && cd data/ 
# # download ModelNet40 from https://mega.nz/file/mm5FhJ7I#jGECWn-QSCLH9LLoxhZzSWnf9LCtCavV12toj9SJKPM 
# # download ScanObjectNN from https://mega.nz/file/ampg2QyT#Exo22r-8jzgCa2MOqoqipd39HVqYKG5iykJ5bovjsuI 

# sudo rmmod nvidia_uvm
# sudo modprobe nvidia_uvm

/bin/bash: ligne 1 : cd: data/: Aucun fichier ou dossier de ce nom


## Depenenancies

In [2]:
import sys
import os
import tqdm
import time 
import json
import matplotlib.pyplot as plt
from datetime import datetime
import subprocess

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
print(torch.cuda.is_available())
from mvtorch.data import ScanObjectNN, CustomDataLoader, ModelNet40
from mvtorch.networks import MVNetwork
from mvtorch.view_selector import MVTN
from mvtorch.mvrenderer import MVRenderer

from utils import *

/home/mpelissi/anaconda3/envs/mvtorchenv1/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


## Config variables

In [3]:
# Run the command and capture the output
current_dir = subprocess.check_output('pwd', shell=True).decode('utf-8').strip()
Papier_inter3_dir = os.path.abspath(os.path.join(current_dir, "../../"))

path_to_Dataset = Papier_inter3_dir+"/Dataset/"
results_dir = current_dir+"/results/"

In [4]:
data_dir= path_to_Dataset+'ModelNet40'#+'ScanObjectNN' # specifiy where did you put the data rel
nb_views = 8 # Number of views generated by view selector
epochs = 100

## Create dataset and dataloader

In [5]:
# dset_train = ScanObjectNN(data_dir=data_dir, split='train')
# dset_test = ScanObjectNN(data_dir=data_dir, split='test')

dset_train = ModelNet40(data_dir=data_dir, split='train')
dset_test = ModelNet40(data_dir=data_dir, split='test')

train_loader = CustomDataLoader(dset_train, batch_size=16, shuffle=True, drop_last=False)
test_loader = CustomDataLoader(dset_test, batch_size=16, shuffle=False, drop_last=False)

In [6]:
len(dset_train), len(dset_test)

(9843, 2468)

## define main components 

In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Create backbone multi-view network (ResNet18)
mvnetwork = MVNetwork(num_classes=len(dset_train.classes), num_parts=None, mode='cls', net_name='resnet18').cuda()

# Create backbone optimizer
optimizer = torch.optim.AdamW(mvnetwork.parameters(), lr=0.00001, weight_decay=0.03)

# Create view selector
mvtn = MVTN(nb_views=nb_views).cuda()

# Create optimizer for view selector (In case views are not fixed, otherwise set to None)
#mvtn_optimizer = torch.optim.AdamW(mvtn.parameters(), lr=0.0001, weight_decay=0.01)
mvtn_optimizer = None

# Create multi-view renderer
mvrenderer = MVRenderer(nb_views=nb_views, return_mapping=False)

# Create loss function for training
criterion = torch.nn.CrossEntropyLoss()

Using cache found in /home/mpelissi/.cache/torch/hub/pytorch_vision_v0.8.2


## train/test loop

In [8]:
## Training
# Create a directory with the current date and time
current_time = datetime.now().strftime("%m-%d_%Hh%Mm%S")
results_dir_current = results_dir+f'results_{current_time}'
os.makedirs(results_dir_current, exist_ok=True)
os.makedirs(results_dir_current+"/best", exist_ok=True)
os.makedirs(results_dir_current+"/by_epoch", exist_ok=True)

# Variables to track the best accuracy
best_accuracy = 0.0
best_epoch = 0

# Lists to store loss and accuracy for plotting
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

# Dictionary to store training parameters and results
training_info = {
    'epochs': epochs,
    'learning_rate': 0.00001,
    'weight_decay': 0.03,
    'batch_size': None,  # You can set this based on your train_loader
    'train_losses': [],
    'train_accuracies': [],
    'test_losses': [],
    'test_accuracies': [],
    'best_accuracy': best_accuracy,
    'best_epoch': best_epoch
}

###############################################################################################
for epoch in range(epochs):
    #break
    #print(f"\nEpoch {epoch + 1}/{epochs}")
    #print("\nTraining...")
    
    running_loss = 0
    correct = 0.0
    
    mvnetwork.train(); mvtn.train(); mvrenderer.train()    
    
    # Use tqdm to create a progress bar
    with tqdm.tqdm(total=len(train_loader), desc=f'Training Epoch {epoch + 1}/{epochs}', unit='batch') as pbar:
        for i, (targets, meshes, points) in tqdm.tqdm(enumerate(train_loader)):      
            azim, elev, dist = mvtn(points, c_batch_size=len(targets))
            rendered_images, _ = mvrenderer(meshes, points, azim=azim, elev=elev, dist=dist)
            outputs = mvnetwork(rendered_images)[0]

            loss = criterion(outputs, targets.cuda())
            running_loss += loss.item()
            loss.backward()
            correct += (torch.max(outputs, dim=1)[1] == targets.cuda()).to(torch.int32).sum().item()
            
            optimizer.step()
            optimizer.zero_grad()
            
            if mvtn_optimizer is not None:
                mvtn_optimizer.step()
                mvtn_optimizer.zero_grad()
                
            print(i, len(targets), (i + 1) * len(targets), correct)

            # Update tqdm progress bar
            pbar.set_postfix({'loss': running_loss / (i + 1), 'accuracy': 100.0 * correct / ((i + 1) * len(targets))})
            #if i == len(train_loader)-1:  
                #pbar.set_postfix({'loss': running_loss / len(train_loader), 'accuracy': 100.0*correct / len(dset_train)})
            pbar.update(1)  # Increment progress bar
    
    avg_train_loss = running_loss / len(train_loader)
    avg_train_accuracy = 100.0 * correct / len(dset_train)
    train_losses.append(avg_train_loss)
    train_accuracies.append(avg_train_accuracy)
        
    #print(f"\nAverage Training Loss = {(running_loss / len(train_loader)):.5f}. Average Training Accuracy = {(100.0*correct / len(dset_train)):.2f}.")
   
    
    ############################################
    # Validate after each epoch
    #print("\nTesting...")
    mvnetwork.eval(); mvtn.eval(); mvrenderer.eval()
    running_loss = 0
    correct = 0.0
    # Use tqdm to create a progress bar
    with tqdm.tqdm(total=len(train_loader), desc=f'Testing Epoch {epoch + 1}/{epochs}', unit='batch') as pbar:
        for i, (targets, meshes, points) in enumerate(test_loader):
            with torch.no_grad():
                azim, elev, dist = mvtn(points, c_batch_size=len(targets))
                rendered_images, _ = mvrenderer(meshes, points, azim=azim, elev=elev, dist=dist)
                outputs = mvnetwork(rendered_images)[0]
                
                loss = criterion(outputs, targets.cuda())
                running_loss += loss.item()
                correct += (torch.max(outputs, dim=1)[1] == targets.cuda()).to(torch.int32).sum().item()

                # Update tqdm progress bar
                #pbar.set_postfix({'loss': running_loss / (i + 1), 'accuracy': 100.0 * correct / ((i + 1) * len(targets))})
                pbar.update(1)  # Increment progress bar
    
    avg_test_loss = running_loss / len(test_loader)
    avg_test_accuracy = 100.0 * correct / len(dset_test)
    test_losses.append(avg_test_loss)
    test_accuracies.append(avg_test_accuracy)
    
    print(f"Total Average Test Loss = {(running_loss / len(test_loader)):.5f}.  Average Test Accuracy = {(100.0*correct / len(dset_test)):.2f}.")
    #pbar.set_postfix({'loss': running_loss / len(test_loader), 'accuracy': 100.0*correct / len(dset_test)})
    
    ############################################
    #print("\Saving...")
    if False:
        # Save the weights after each epoch
        torch.save(mvnetwork.state_dict(), results_dir_current+f'/by_epoch/mvnetwork_epoch_{epoch + 1}.pth')
        torch.save(mvtn.state_dict(), results_dir_current+f'/by_epoch/mvtn_epoch_{epoch + 1}.pth')
        torch.save(mvrenderer.state_dict(), results_dir_current+f'/by_epoch/mvrenderer_epoch_{epoch + 1}.pth')
        
        # Save the weights if the current epoch's accuracy is better than the best accuracy
        test_accuracy = 100.0 * correct / len(dset_test)
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            best_epoch = epoch + 1  # Store the best epoch (1-indexed)
            
            # Save the weights of the best epoch
            torch.save(mvnetwork.state_dict(), results_dir_current+f'/best/mvnetwork_best_{epoch+1}.pth')
            torch.save(mvtn.state_dict(), results_dir_current+f'/best/mvtn_best_{epoch+1}.pth')
            torch.save(mvrenderer.state_dict(), results_dir_current+f'/best/mvrenderer_best_{epoch+1}.pth')
            print(f"Best epoch saved : {epoch+1}, with accuracy: {best_accuracy:.2f}%")
            
        # Update training_info with the latest epoch results
        training_info['train_losses'].append(avg_train_loss)
        training_info['train_accuracies'].append(avg_train_accuracy)
        training_info['test_losses'].append(avg_test_loss)
        training_info['test_accuracies'].append(avg_test_accuracy)
        training_info['best_accuracy'] = best_accuracy
        training_info['best_epoch'] = best_epoch
        
        # Save training information to a JSON file
        with open(results_dir_current+'/training_info.json', 'w') as json_file:
            json.dump(training_info, json_file, indent=4)


###############################################################################################
print("Training complete! Best epoch:", best_epoch)

# Save the training information with 2 figure
save_loss_acc(results_dir_current,train_losses, test_losses, train_accuracies, test_accuracies)

Training Epoch 1/100:   0%|          | 1/616 [00:07<1:18:58,  7.71s/batch, loss=3.88, accuracy=6.25]

0 16 16 1.0


Training Epoch 1/100:   0%|          | 2/616 [00:11<54:29,  5.33s/batch, loss=3.85, accuracy=3.12]  

1 16 32 1.0


Training Epoch 1/100:   0%|          | 3/616 [00:15<50:07,  4.91s/batch, loss=3.82, accuracy=4.17]

...

469 16 7520 3662.0


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7302cd682370>>
Traceback (most recent call last):
  File "/home/mpelissi/anaconda3/envs/mvtorchenv1/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Training Epoch 1/100:  76%|███████▋  | 471/616 [40:27<19:00,  7.86s/batch, loss=2.16, accuracy=48.7]

470 16 7536 3671.0


Training Epoch 1/100:  77%|███████▋  | 472/616 [40:30<15:04,  6.28s/batch, loss=2.16, accuracy=48.7]

471 16 7552 3681.0


## Profiler

In [8]:
## Opitmize code 1/3
train_loader = CustomDataLoader(dset_train, batch_size=20, shuffle=True, drop_last=False, pin_memory=True)
test_loader = CustomDataLoader(dset_test, batch_size=20, shuffle=False, drop_last=False, pin_memory=True)

In [ ]:
import torch
from torch.autograd import profiler as old_profiler

for epoch in range(epochs):         
    # Set up profiler to analyze one epoch
    with old_profiler.profile(enabled=True, use_cuda=True) as prof: 
         # Training loop
        running_loss = 0
        correct = 0.0        
        mvnetwork.train(); mvtn.train(); mvrenderer.train()    
        
        # Use tqdm to create a progress bar
        with tqdm.tqdm(total=len(train_loader), desc=f'Training Epoch {epoch + 1}/{epochs}', unit='batch') as pbar:
            for i, (targets, meshes, points) in tqdm.tqdm(enumerate(train_loader)):  
                ## Opitmize code 2/3 
                # Move data to GPU with non-blocking transfer
                # targets = targets.cuda(non_blocking=True)
                # points = points.cuda(non_blocking=True)
                # meshes = torch.tensor(meshes).cuda(non_blocking=True)
                                  
                azim, elev, dist = mvtn(points, c_batch_size=len(targets))
                rendered_images, _ = mvrenderer(meshes, points, azim=azim, elev=elev, dist=dist)
                outputs = mvnetwork(rendered_images.cuda())[0]

                loss = criterion(outputs, targets.cuda())
                #loss = criterion(outputs, targets) ## Opitmize code 3/3
                running_loss += loss.item()
                loss.backward()
                correct += (torch.max(outputs, dim=1)[1] == targets.cuda()).to(torch.int32).sum().item()
                
                optimizer.step()
                optimizer.zero_grad()
                
                if mvtn_optimizer is not None:
                    mvtn_optimizer.step()
                    mvtn_optimizer.zero_grad()

                # Update tqdm progress bar
                pbar.set_postfix({'loss': running_loss / (i + 1), 'accuracy': 100.0 * correct / ((i + 1) * len(targets))})
                pbar.update(1)  # Increment progress bar
        
        avg_train_loss = running_loss / len(train_loader)
        avg_train_accuracy = 100.0 * correct / len(dset_train)
        print(f"Training Loss: {avg_train_loss:.4f}, Accuracy: {avg_train_accuracy:.2f}%")
        
        ############################################
       
    # Print the profiling results after each epoch
    print('Profiling results:')
    print(prof.key_averages().table(sort_by="self_cpu_time_total"))          

    if epoch == 0:
        break

## Inference

In [ ]:
# model = TheModelClass(*args, **kwargs)
# model.load_state_dict(torch.load(PATH, weights_only=True))
# model.eval()

# Assuming you have defined your models (mvnetwork, mvtn, mvrenderer) again
mvnetwork.load_state_dict(torch.load(results_dir+f'best/mvnetwork_best.pth', weights_only=True)); mvnetwork.eval()
mvtn.load_state_dict(torch.load(results_dir+f'best/mvtn_best.pth', weights_only=True)); mvtn.eval()
mvrenderer.load_state_dict(torch.load(results_dir+f'best/mvrenderer_best.pth', weights_only=True)); mvrenderer.eval()


In [ ]:
# Test the network
print("\n\nTesting...")
mvnetwork.eval(); mvtn.eval(); mvrenderer.eval()
running_loss = 0
correct = 0.0
# Use tqdm to create a progress bar
with tqdm.tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{epochs}', unit='batch') as pbar:
    for i, (targets, meshes, points) in enumerate(test_loader):
        with torch.no_grad():
            start_time = time.time()  # Start timing the batch processing
            
            azim, elev, dist = mvtn(points, c_batch_size=len(targets))
            rendered_images, _ = mvrenderer(meshes, points, azim=azim, elev=elev, dist=dist)
            outputs = mvnetwork(rendered_images)[0]
            
            loss = criterion(outputs, targets.cuda())
            running_loss += loss.item()
            correct += (torch.max(outputs, dim=1)[1] == targets.cuda()).to(torch.int32).sum().item()

            # Update tqdm progress bar
            elapsed_time = time.time() - start_time
            pbar.set_postfix({'loss': running_loss / (i + 1), 'accuracy': 100.0 * correct / ((i + 1) * len(targets))})
            pbar.update(1)  # Increment progress bar
            
print(f"\nTotal Average Test Loss = {(running_loss / len(test_loader)):.5f}.  Average Test Accuracy = {(100.0*correct / len(dset_test)):.2f}.")

# Convert

In [ ]:
#jupyter nbconvert --to script classification.ipynb --output tmp_classification.py
